In [ ]:
using CSV
using DataFrames
using Plots
using Statistics
using Flux
using Random
using StatsBase
using Dates

In [ ]:
data = CSV.read("datasets/Data 4.csv", DataFrame)
rename!(data, [:tanggal, :X1, :X2, :X3, :Y, :X4, :X5]);

In [ ]:
data.S = repeat(1.0:12.0,20)
data

In [ ]:
yd = data.Y
xd = select(data, [:X1, :X2, :X3, :X4, :X5, :S]);

In [ ]:
xs = Matrix(Matrix(xd)');
ys = Matrix(yd');

In [ ]:
#Flux.normalise(xs)

### Model Training

In [ ]:
# 1 Hidden Layer Architecture
function model1(node)
    model = Chain(
        BatchNorm(6),
        Dense(6,node),
        Dense(node,1,relu)
    )
    return model
end

In [ ]:
# 2 Hidden Layers Architecture
function model2(node)
    model = Chain(
        BatchNorm(6),
        Dense(6,node),
        Dense(node,node),
        Dense(node,1,relu)
    )
    return model
end

In [ ]:
# Loss function
L(x,y) = Flux.Losses.mae(model(x), y);

In [ ]:
# Optimizer
η = 0.5
opt = NADAM(η)

In [ ]:
# First training (without cross-validation)
xtrain = Array{Float64}(undef,6,228)
ytrain = Array{Int64}(undef,1,228)

sub_test = 19
itest = collect(1:12) .+ sub_test*12
itrain = setdiff(1:240,itest)
xtrain = xs[:,itrain]
ytrain = ys[:,itrain]
xtest = xs[:,itest]
ytest = ys[:,itest]
max_epoch = 1000;

# For 1 hidden layer: use model1()
# For 2 hidden layers: use model2()
model = model2(3)
θ = Flux.params(model);
# Train model
loss_train = zeros(max_epoch)

for epoch in 1:max_epoch
    Flux.train!(L, θ, [(xtrain, ytrain)], opt)
    loss_train[epoch] = L(xtrain, ytrain)
    rmse_train = sqrt(L(xtrain,ytrain))
    epoch%100==0 ? println("training error after epoch $epoch: $rmse_train") : continue
end

In [ ]:
dts = Date(2001,1):Month(1):Date(2001,1)+Month(227)
tick_years = Date.([2001, 2010, 2019])
DateTick = Dates.format.(tick_years, "yyyy")

plot(dts, [ytrain' model(xtrain)'], label=["Nilai Aktual" "Nilai Prediksi"],linewidth=1.5, xticks=false)
plot!(xticks=(tick_years,DateTick), xtickfontsize=8)
plot!(ylims=(-200,11000))
xlabel!("Periode")
ylabel!("Jumlah Hotspot")
#savefig("train.svg")

In [ ]:
dts = Date(2020,1):Month(1):Date(2020,1)+Month(11)
tick_years = [Date(2020,1),Date(2020,6),Date(2020,12)]
DateTick = Dates.format.(tick_years, "u-yyyy")

plot(dts, [ytest' model(xtest)'], label=["Nilai Aktual" "Nilai Prediksi"],linewidth=1.5, xticks=false)
plot!(xticks=(tick_years,DateTick), xtickfontsize=7)
plot!(ylims=(-5,190))
xlabel!("Periode")
ylabel!("Jumlah Hotspot")
#savefig("test.svg")

In [ ]:
rmse(x,xduga) = sqrt(mean((xduga .- x).^2));
rmse(ytest', model(xtest)')

In [ ]:
rsquare(y,yduga) = 1-(sum((y.-yduga).^2))./(sum((y.-mean(y)).^2));
rsquare(ytest', model(xtest)')

In [ ]:
# Training with cross-validation
xtrain = Array{Float64}(undef,6,228)
ytrain = Array{Int64}(undef,1,228)
model = []

for node in 3:12 # -> Combination of the hidden node
    println("Hidden node: $node")
    
    # Repitition
    for i in 1:10
        train_result = Array{Float64}(undef,1,240)
        max_epoch = 1000
        println("Iteration $i")
        
        # Loop for cross validation
        for sub_test in 0:19
            itest = collect(1:12) .+ sub_test*12
            itrain = setdiff(1:240,itest)
            xtrain = xs[:,itrain]
            ytrain = ys[:,itrain]
            xtest = xs[:,itest]
            ytest = ys[:,itest]
            # For 1 hidden layer: use model1()
            # For 2 hidden layers: use model2()
            model = model2(node)
            θ = Flux.params(model);
            # Train model
            loss_train = zeros(max_epoch)
            for epoch in 1:max_epoch
                Flux.train!(L, θ, [(xtrain, ytrain)], opt)
            end
            train_result[itest] = model(xtest)'
        end
        
        mae(y,yduga) = mean(abs.(yduga .- y));
        cross_val_mae = mae(ys', train_result')
        println("mae: $cross_val_mae")
        #rmse(y,yduga) = sqrt(mean((yduga .- y).^2));
        #cross_val_rmse = rmse(ys', train_result')
        #println("rmse: $cross_val_rmse")
        rsquare(y,yduga) = 1-(sum((y.-yduga).^2))./(sum((y.-mean(y)).^2));
        cross_val_rsquare = rsquare(ys', train_result')
        println("r-square: $cross_val_rsquare")
        println("-----------------------------")
    end
    
    println("")
    
end

In [ ]:
train_result

In [ ]:
dts = Date(2001,1):Month(1):Date(2001,1)+Month(239)
tick_years = Date.([2001, 2010, 2020])
DateTick = Dates.format.(tick_years, "yyyy")

plot(dts, [ys' train_result'], label=["Nilai Aktual" "Nilai Prediksi"],linewidth=1.5, xticks=false)
plot!(xticks=(tick_years,DateTick), xtickfontsize=8)
plot!(ylims=(-200,11000))
xlabel!("Periode")
ylabel!("Jumlah Hotspot")
#savefig("cross.svg")

### Feature Importance

In [ ]:
# Best Model Architecture
function modelfix()
    model = Chain(
        BatchNorm(6),
        Dense(6,5),
        Dense(5,1,relu)
    )
    return model
end

In [ ]:
# Loss function MAE
L(x,y) = Flux.Losses.mae(model(x), y);

In [ ]:
# Optimizer
η = 0.01
opt = NADAM(η)

In [ ]:
model = modelfix();

In [ ]:
# Fit the model once

xtrain = Array{Float64}(undef,6,228)
ytrain = Array{Int64}(undef,1,228)

train_result = Array{Float64}(undef,1,240)
max_epoch = 1000
for sub_test in 0:19
    itest = collect(1:12) .+ sub_test*12
    itrain = setdiff(1:240,itest)
    xtrain = xs[:,itrain]
    ytrain = ys[:,itrain]
    xtest = xs[:,itest]
    ytest = ys[:,itest]
    model = modelfix()
    θ = Flux.params(model);
    # Train model
    loss_train = zeros(max_epoch)
    for epoch in 1:max_epoch
        Flux.train!(L, θ, [(xtrain, ytrain)], opt)
    end
    train_result[itest] = model(xtest)'
end
mae(y,yduga) = mean(abs.(yduga .- y));
cross_val_mae = mae(ys', train_result')
println("mae: $cross_val_mae")
rsquare(y,yduga) = 1-(sum((y.-yduga).^2))./(sum((y.-mean(y)).^2));
cross_val_rsquare = rsquare(ys', train_result')
println("r-square: $cross_val_rsquare")
println("-----------------------------")

In [ ]:
# Estimate the error change

dummy = copy(data)
model_mae = 315.825149675297
model_rsquare = 0.761082061725953

for i in 1:10
    # Change the variable here
    #dummy.X1 = rand(minimum(data.X1):maximum(data.X1), 240)
    dummy.X1 = shuffle!(data.X1)
    ym = dummy.Y
    xm = select(dummy, [:X1, :X2, :X3, :X4, :X5, :S])
    xn = Matrix(Matrix(xm)')
    yn = Matrix(ym');
    println("Iteration $i")
    
    mae(y,yduga) = mean(abs.(yduga .- y));
    cross_val_mae = mae(yn', model(xn)')
    mae_increase = cross_val_mae - model_mae
    println("mae increase: $mae_increase")
    rsquare(y,yduga) = 1-(sum((y.-yduga).^2))./(sum((y.-mean(y)).^2));
    cross_val_rsquare = rsquare(yn', model(xn)')
    rsquare_change = abs(cross_val_rsquare - model_rsquare)
    println("r-square change: $rsquare_change")
    println("-----------------------------------")
end